In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

In [3]:
# Predict

prompt = PromptTemplate.from_template(
    "What is the advantage of {frontend_stack}?"
).format(frontend_stack="Svelte")
chat.predict(prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a frontend expert. Also, you only reply in {language}"),
        ("ai", "{tone}"),
        ("human", "What is the advantage of {frontend_stack}? Also, what's your name?"),
    ]
).format_messages(language="Korean", tone="안녕하세요!", frontend_stack="Next.js")
chat.predict_messages(prompt)

Svelte offers several advantages over other front-end frameworks:

1. Performance: Svelte compiles the code during build time, resulting in a small and efficient JavaScript bundle. It eliminates overhead and reduces the size of the application, leading to faster loading times and improved performance.

2. Small footprint: Svelte has a small footprint, which means it requires fewer dependencies and less code compared to other frameworks. This makes it ideal for projects with limited resources or for building fast, lightweight applications.

3. Reactive updates: Svelte uses a reactive programming model, where changes in the application state automatically update the DOM. This approach eliminates the need for a virtual DOM, resulting in faster rendering and a more efficient user interface.

4. Easy learning curve: Svelte has a simple and intuitive syntax, making it easy to learn for developers who are new to front-end development or coming from other frameworks. Its approach is closer to 

AIMessageChunk(content='Next.js의 장점은 다음과 같습니다:\n\n1. 서버 사이드 렌더링 (SSR): Next.js는 서버에서 페이지를 렌더링하기 때문에 초기 로딩 속도가 빠르고 SEO에 이점이 있습니다.\n\n2. 정적 사이트 생성 (SSG): Next.js는 사전에 빌드된 정적 파일을 제공하여 성능을 향상시킵니다.\n\n3. 자동 코드 분할: Next.js는 페이지별로 자동으로 코드를 분할하여 최적의 번들링을 제공하므로 초기 로딩 시간을 최소화합니다.\n\n4. 개발자 경험: Next.js는 기본적으로 프레임워크와 라우터를 제공하여 개발 프로세스를 단순화하고 생산성을 높입니다.\n\n저는 AI로 구축된 가상 도우미이기 때문에 개인의 이름을 가지고 있지 않습니다. 다른 질문이 있으면 언제든지 물어보세요!')

In [4]:
# Output Parser

from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        return list(map(str.strip, text.split(",")))


parser = CommaOutputParser()
parser.parse(" Hello, how,are,you ")

['Hello', 'how', 'are', 'you']

In [5]:
# Langchain Expression Language

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else",
        ),
        ("human", "{question}"),
    ]
)
chain = prompt | chat | parser
chain.invoke(
    {"max_items": 5, "question": "Interesting things about frontend development"}
)

html, css, javascript, responsive design, user experience

['html', 'css', 'javascript', 'responsive design', 'user experience']

In [6]:
# Chaining Chains

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it.",
        ),
        ("human", "{recipe}"),
    ]
)
veg_chef_chain = veg_chef_prompt | chat

chain = {"recipe": chef_chain} | veg_chef_chain
chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Here's a recipe for a classic Indian dish called Butter Chicken:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 2 tablespoons butter
- 1 tablespoon vegetable oil
- 1 onion, finely chopped
- 2 teaspoons garlic paste
- 2 teaspoons ginger paste
- 2 teaspoons garam masala (Indian spice blend)
- 1 teaspoon turmeric powder
- 1 teaspoon chili powder (adjust according to your spice preference)
- 1 cup tomato puree
- 1 cup heavy cream
- Salt to taste
- Fresh cilantro leaves for garnish

Instructions:
1. Heat the butter and oil in a large pan over medium heat. Add the chopped onion and sauté until golden brown.
2. Add the garlic and ginger paste to the pan and cook for another minute until fragrant.
3. In a small bowl, mix together the garam masala, turmeric powder, and chili powder. Add this spice mixture to the pan and cook for a minute, stirring constantly.
4. Add the chicken pieces to the pan

AIMessageChunk(content="For a vegetarian version of Butter Chicken, you can replace the chicken with a vegetarian protein alternative such as tofu or paneer (Indian cottage cheese). Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 2 tablespoons butter\n- 1 tablespoon vegetable oil\n- 1 onion, finely chopped\n- 2 teaspoons garlic paste\n- 2 teaspoons ginger paste\n- 2 teaspoons garam masala (Indian spice blend)\n- 1 teaspoon turmeric powder\n- 1 teaspoon chili powder (adjust according to your spice preference)\n- 1 cup tomato puree\n- 1 cup heavy cream (you can substitute with coconut milk for a vegan version)\n- Salt to taste\n- Fresh cilantro leaves for garnish\n\nInstructions:\n1. Heat the butter and oil in a large pan over medium heat. Add the chopped onion and sauté until golden brown.\n2. Add the garlic and ginger paste to the pan and cook for another minute until fragrant.\n3. In a small bowl, mix together the garam masal

In [7]:
# FewShotPromptTemplate, Example Selector

from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


template = "Human: {question}\nAI: {answer}"
prompt = PromptTemplate.from_template(template)
selector = RandomExampleSelector(examples=examples)
prompt = FewShotPromptTemplate(
    example_prompt=prompt,
    example_selector=selector,
    suffix="Human: {question}",
    input_variables=["question"],
)

chain = prompt | chat
chain.invoke({"question": "I wonder about Sweden"})

AI: 
Sweden is a Nordic country located in Northern Europe. Here are some facts about Sweden:

Capital: Stockholm
Language: Swedish
Currency: Swedish Krona
Government: Constitutional monarchy
Population: Approximately 10 million
Famous for: Viking history, beautiful landscapes, IKEA, ABBA, and Swedish meatballs
Notable landmarks: Stockholm Palace, Vasa Museum, and the Icehotel in Jukkasjärvi
Interesting fact: Sweden is known for its high standard of living and is consistently ranked as one of the world's happiest countries.

AIMessageChunk(content="AI: \nSweden is a Nordic country located in Northern Europe. Here are some facts about Sweden:\n\nCapital: Stockholm\nLanguage: Swedish\nCurrency: Swedish Krona\nGovernment: Constitutional monarchy\nPopulation: Approximately 10 million\nFamous for: Viking history, beautiful landscapes, IKEA, ABBA, and Swedish meatballs\nNotable landmarks: Stockholm Palace, Vasa Museum, and the Icehotel in Jukkasjärvi\nInteresting fact: Sweden is known for its high standard of living and is consistently ranked as one of the world's happiest countries.")

In [8]:
# FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "country": "Italy",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "country": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers"),
        prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = prompt | chat
chain.invoke({"country": "Sweden"})

I know this:
Capital: Stockholm
Language: Swedish
Food: Meatballs and Herring
Currency: Swedish Krona

AIMessageChunk(content='I know this:\nCapital: Stockholm\nLanguage: Swedish\nFood: Meatballs and Herring\nCurrency: Swedish Krona')

In [11]:
# Load Prompt

from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")
chain = prompt | chat
chain.invoke({"country": "South Korea"})

The capital of South Korea is Seoul.

AIMessageChunk(content='The capital of South Korea is Seoul.')